In [1]:
import numpy as np
import pandas as pd
import ast

from gensim import corpora
import gensim
import pyLDAvis.gensim

from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

import time
from tqdm.notebook import tqdm
import warnings

import matplotlib.pyplot as plt


In [2]:
food = pd.read_csv("./data/food_topic.csv", encoding = 'utf-8-sig')

In [3]:
food.head()

,index,SAMPLE_ID,식품코드,DB군,상용제품,food_name,연도,지역 / 제조사,채취시기,식품상세분류,...,식염상당량(g),회분(g),폐기율(%),가식부(%),산가용성물질(%),카페인(㎎),성분표출처,발행기관,topic,topic_rate
0,43,D000157-ZZ-D08,D000157,음식,품목대표,열무김치국수,2019,전국(대표),8월,면류,...,0,10.3,0,0,0,0,식약처('17) 제5권,식품의약품안전처,1,0.7678
1,44,D000160-103-D08,D000160,음식,품목대표,올갱이국수,2019,정선,8월,면류,...,0,10.9,0,0,0,0,식약처('16) 제4권,식품의약품안전처,1,0.7678
2,48,D018170-ZZ-AVG,D018170,음식,품목대표,바지락칼국수,2020,전국(대표),평균,면류,...,0,6.1,0,0,0,0,식약처 영양실태조사('20),식품의약품안전처,1,0.7678
3,53,D018178-ZZ-AVG,D018178,음식,품목대표,잔치국수,2020,전국(대표),평균,면류,...,0,4.6,0,0,0,0,식약처 영양실태조사('20),식품의약품안전처,1,0.7678
4,54,D018179-ZZ-AVG,D018179,음식,품목대표,쟁반국수,2020,전국(대표),평균,면류,...,0,8.7,0,0,0,0,식약처 영양실태조사('20),식품의약품안전처,1,0.7678


In [9]:
youtube = pd.read_csv("./data/youtube_2.csv", encoding = 'utf-8')

In [10]:
youtube.head()

,Column1,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,category
0,1656,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12,UCRuSxVu4iqTK5kCh90ntAgA,총몇명,1,2020-08-12,총몇명|재밌는 만화|부락토스|루시퍼|총몇명 프리퀄|총몇명 스토리,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",Film & Animation
1,1891,Q9t9fl3vHl0,불사신들을 너무 많이 생산해버린 미국 제약 회사,2020-09-09,UCxlv4aOnrRTXMRSL8bVJqEw,B Man 삐맨,1,2020-09-11,더보이즈|더보이즈 시즌1|더보이즈 시즌2|미드 더보이즈|미드 더보이즈 시즌2|미드 ...,504063,4712,198,391,https://i.ytimg.com/vi/Q9t9fl3vHl0/default.jpg,False,False,여러분 안녕하세요 삐맨입니다.오늘은 '더 보이즈' 시즌 2에 대해 알아보려 합니다....,Film & Animation
2,1890,ny5ZVoC-pp4,"승찬, 우정을 느끼다 [총몇명 프리퀄]",2020-09-09,UCRuSxVu4iqTK5kCh90ntAgA,총몇명,1,2020-09-11,총몇명|유승찬|민모리|ㅋㅋㅋ|웃긴|부락토스,1548169,38486,684,4607,https://i.ytimg.com/vi/ny5ZVoC-pp4/default.jpg,False,False,"오늘도 역시 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",Film & Animation
3,1889,sIzWUz6OnG8,"묵직하고, 치밀하며, 몰입도 극강인 갓띵작 스릴러 (결말포함)",2020-09-04,UCOXAccjfcIvd-uRQgK2Fsrw,늘보Movie,1,2020-09-10,영화리뷰|늘보무비|인생영화|명작리뷰|프리즈너스,432102,4503,128,293,https://i.ytimg.com/vi/sIzWUz6OnG8/default.jpg,False,False,"프리즈너스 (Prisoners, 2013)영화 프리즈너스 소개영상입니다 :)#늘보무...",Film & Animation
4,1888,c4l8OGyj-lg,모두가 솔직하면 무서운 세상이 펼쳐진다 ㅋㅋㅋㅋ,2020-09-05,UCb_GEWOGkCTXawWZKDtALwg,9bul,1,2020-09-10,구불|거짓말|거짓말의 발명|병맛영화|영화추천,522541,9677,184,1301,https://i.ytimg.com/vi/c4l8OGyj-lg/default.jpg,False,False,메일주소 9bul.contact@gmail.com 공식 페이스북 페이지https:/...,Film & Animation
